In [ ]:
# using python 3.9.7
# analysis script for looking for single or multi photon events in the photon detectors.
# The nois

In [1]:
from os import listdir
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# acquire all file names in directory
filespath = '/Users/samuelfogarty/OneDrive/Desktop/OneDrive/photondetectortests/dune_fd_pds_data/singlephotonsearch/'
#abovebreakdown = '12usecwindow__25MHzSR_1kpts_10mVperdiv_ch4_44.135Bias'
#belowbreakdown = '12usecwindow_25MHzSR_1kpts_10mVperdiv_ch4_10.932VBias'
abovebreakdown = '10mswindow_25MHzSR_1kpts_10mVperdiv_ch4_44.135VBias'
belowbreakdown = '10mswindow_25MHzSR_1kpts_10mVperdiv_ch4_10.935VBias'

# file path of each directory
filespath_bbd = filespath + belowbreakdown
filespath_abd = filespath + abovebreakdown
# list of file names in each directory
filenames_bbd = os.listdir(filespath_bbd)
filenames_abd = os.listdir(filespath_abd)

In [3]:
def readfile(filepath):
    word,words = 0,0
    ts, Vs1 = [], [] # seconds, volts


    with open(filepath) as fp: # this line opens the file and makes sure to close it when all is done
        line = fp.readline()
        while line: # start read file loop
            words = line.strip().split(',') # splits each line at a comma
                                                # the strip part makes sure there aren't any weird characters left over at the start or end of a line
            if (words[0] == 'TIME' and words[0] != ''): # finds where to start taking the data, may need to change 
                word = 1                                # for different files
                line = fp.readline() # get next line
                words = line.strip().split(',')
            if word == 1 and words[0] != '': # this ensures it starts recording the data lines only 
                ts.append(words[0].strip())
                Vs1.append(words[1].strip()) 
            line = fp.readline()
    return ts, Vs1


In [ ]:
t_window = 1 # desired window size; microseconds
V_threshold = 2 # nVs
t_buffer = 3 # usec

In [8]:
numfiles = len(filenames_abd)
#filepath_bbd = filespath_bbd + '/' + filenames_bbd[0]
filepath_abd = filespath_abd + '/' + filenames_abd[0]
#ts,Vs1 = readfile(filepath_bbd)
ts,Vs1 = readfile(filepath_abd)
# make into array and cast as floats
ts = np.array(ts, dtype='float')
Vs1 = np.array(Vs1, dtype='float')

In [18]:
# function to search for single or multi photon events
def findcandidates(ts, Vs1):
    t_spacing = ts[1] - ts[0]
    npoints_window = int(t_window*1e-6/t_spacing) # number of points in window
    photon_candidates = []
    # roll through data with a specified time window, take integrals and look for single photon candidates
    for index_start in range(0, len(ts) - npoints_window):
        ts_sel = ts[index_start:(index_start + npoints_window)]
        Vs1_sel = Vs1[index_start:(index_start + npoints_window)]
        trace_integral = np.trapz(Vs1_sel, x=ts_sel)
        if trace_integral > V_threshold*1e-9:
            photon_candidates.append(trace_integral)
    return photon_candidates


In [19]:
# get times and voltages for a specified csv file
def getData(filepath):
    times,voltages = readfile(filepath)
    ts = np.array(times, dtype='float')
    Vs1 = np.array(voltages, dtype='float')
    return ts, Vs1

In [21]:
# loop through each file, above and below breakdown
ncandidates_abd = 0
ncandidates_bbd = 0
for filename_abd in filenames_abd:
    filepath_abd = filespath_abd + '/' + filename_abd
    t, V = getData(filepath_abd)
    ncandidates_abd += len(findcandidates(t,V))
for filename_bbd in filenames_bbd:
    filepath_bbd = filespath_bbd + '/' + filename_bbd
    t, V = getData(filepath_bbd)
    ncandidates_bbd += len(findcandidates(t,V))

/Users/samuelfogarty/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:47: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [23]:
print("number of photon candidates below breakdown = ",ncandidates_bbd)
print("number of photon candidates above breakdown = ",ncandidates_abd)

number of photon candidates below breakdown =  69187
number of photon candidates above breakdown =  135712


In [ ]:
ncandidates